# Create NCBI gene reference files

1. REF_NCBIGENE2LABEL: NCBI Gene ID to gene symbol
2. REF_NCBIGENE2NAMES: NCBI Gene ID to gene name and synonyms
3. REF_NAMES2NCBIGENE: gene name and synonyms to NCBI Gene ID



In [1]:
import urllib.request
import gzip
import compress_pickle
import os
import string
import pandas as pd
import numpy as np
import re
DATA_DIR = '/Users/luna/Desktop/CRBM/AMAS_proj/Data/'
output_dir = '/Users/luna/Desktop/CRBM/AMAS_proj/AAAIM/data/ncbigene/'

## Data

Data are obtained from the NCBI gene FTP site: https://ftp.ncbi.nih.gov/gene/DATA/GENE_INFO/.

In [2]:
# All gene data
input_file = DATA_DIR + "ncbi/All_Data.gene_info.gz"
columns_to_keep = ["#tax_id", "GeneID", "Symbol", "Synonyms","type_of_gene"]
df = pd.read_csv(input_file, compression='gzip', sep='\t',usecols=columns_to_keep)
df.head()

,#tax_id,GeneID,Symbol,Synonyms,type_of_gene
0,7,5692769,NEWENTRY,-,other
1,9,2827857,NEWENTRY,-,other
2,11,10823747,NEWENTRY,-,other
3,14,6951813,NEWENTRY,-,other
4,19,3758873,NEWENTRY,-,other


In [3]:
# Exclude certain types of genes
print(df['type_of_gene'].value_counts())
# exclude_types = ['biological-region', 'unknown', 'other']
# df = df[~df['type_of_gene'].isin(exclude_types) & (df['Symbol'] != 'NEWENTRY')]

# Only include protein-coding genes
include_types = ['protein-coding']
df = df[df['type_of_gene'].isin(include_types)]

type_of_gene
protein-coding       46235835
ncRNA                 3480011
tRNA                  3033345
pseudo                2737966
rRNA                   995495
snRNA                  586439
snoRNA                 505390
biological-region      168474
other                  108372
unknown                 61023
miscRNA                  5128
scRNA                      21
Name: count, dtype: int64


## Create REF_NCBIGENE2LABEL

In [5]:
# all organisms with protein-coding genes
output_gene2label = "ncbigene2label_all_protein-coding.lzma"
gene_id_to_symbol = dict(zip(df['GeneID'], df['Symbol']))
with open(os.path.join(output_dir,output_gene2label), 'wb') as handle:
    compress_pickle.dump(gene_id_to_symbol, handle, compression="lzma", set_default_extension=False)
print(f"GeneID to Symbol saved to {output_dir+output_gene2label}")

GeneID to Symbol saved to /Users/luna/Desktop/CRBM/AMAS_proj/AAAIM/data/ncbi_gene/ncbigene2label_all_protein_coding.lzma


In [6]:
len(gene_id_to_symbol)

46235835

This file is too large, only the dictionary takes 114.7MB. I'll go with the organisms in Bigg models first.

In [7]:
# Bigg organisms with protein-coding genes
bigg_model_dir = '/Users/luna/Desktop/CRBM/AMAS_proj/Models/BiggModels/'
bigg_organisms = pd.read_csv(bigg_model_dir + "bigg_organisms.csv")
taxids = bigg_organisms['tax_id'].unique().tolist()
taxids = [x for x in taxids if x and str(x).strip()] # Remove empty strings

# Only include rows where tax_id is in taxids
df['#tax_id'] = df['#tax_id'].astype(str)
bigg_df = df[df['#tax_id'].isin(taxids)]
bigg_df = bigg_df.map(str)
output_geneinfo_bigg = "ncbi_gene_info_bigg_organisms_protein-coding.csv"
bigg_df.to_csv(output_dir + output_geneinfo_bigg, index=False)

print(f"Number of organisms contained in NCBI gene info: {bigg_df['#tax_id'].nunique()}")
print(f"Number of rows: {len(bigg_df)}")
bigg_df.head()
print(f"All gene info of bigg organisms saved to {output_dir + output_geneinfo_bigg}")

# Create a dictionary of gene IDs to symbols for the bigg organisms
output_gene2label = "ncbigene2label_bigg_organisms_protein-coding.lzma"
gene_id_to_symbol = dict(zip(bigg_df['GeneID'], bigg_df['Symbol']))
with open(os.path.join(output_dir,output_gene2label), 'wb') as handle:
    compress_pickle.dump(gene_id_to_symbol, handle, compression="lzma", set_default_extension=False)
print(f"GeneID to Symbol saved to {output_dir+output_gene2label}")

Number of organisms contained in NCBI gene info: 18
Number of rows: 146842
All gene info of bigg organisms saved to /Users/luna/Desktop/CRBM/AMAS_proj/AAAIM/data/ncbi_gene/ncbi_gene_info_bigg_organisms_protein_coding.csv
GeneID to Symbol saved to /Users/luna/Desktop/CRBM/AMAS_proj/AAAIM/data/ncbi_gene/ncbigene2label_bigg_organisms_protein_coding.lzma


It is 502KB for the Bigg organisms with protein-coding genes, which is much smaller.

## Create REF_NCBIGENE2NAMES

This is used for mapping ncbigene ids to standard name and synonyms, and used in the RAG approach.

In [8]:
output_geneinfo_bigg = "ncbi_gene_info_bigg_organisms_protein-coding.csv"
geneinfo_df = pd.read_csv(output_dir + output_geneinfo_bigg, dtype=str)
geneid_to_names = {}
for idx, row in geneinfo_df.iterrows():
    gene_id = row['GeneID']
    symbol = row['Symbol'].strip() if pd.notnull(row['Symbol']) else ''
    synonyms = row['Synonyms'].strip() if pd.notnull(row['Synonyms']) else ''
    names = []
    if symbol and symbol != '-':
        names.append(symbol)
    if synonyms and synonyms != '-':
        # Split by '|' and strip whitespace
        names.extend([s.strip() for s in synonyms.split('|') if s.strip() and s.strip() != symbol])
    # Remove duplicates while preserving order
    seen = set()
    unique_names = []
    for n in names:
        if n not in seen:
            unique_names.append(n)
            seen.add(n)
    geneid_to_names[gene_id] = unique_names

print(len(geneid_to_names))
geneid_to_names

146842


{'801484': ['cob'],
 '801489': ['rtl'],
 '801491': ['nad2'],
 '801492': ['nad5'],
 '801494': ['nad4'],
 '801495': ['cox1'],
 '801499': ['nad1'],
 '801500': ['nad6'],
 '2716952': ['rps8'],
 '2716955': ['rps4'],
 '2716958': ['psaJ'],
 '2716959': ['atpI'],
 '2716960': ['psbJ'],
 '2716961': ['psbD'],
 '2716962': ['ORF2971'],
 '2716963': ['psbC'],
 '2716966': ['chlB'],
 '2716969': ['psbA'],
 '2716971': ['ycf12'],
 '2716972': ['atpE'],
 '2716973': ['rps7'],
 '2716974': ['rps14'],
 '2716975': ['psbM'],
 '2716976': ['psbZ'],
 '2716978': ['psbK'],
 '2716979': ['tufA'],
 '2716983': ['atpF'],
 '2716986': ['chlN'],
 '2716987': ['psbA'],
 '2716989': ['petA'],
 '2716990': ['psbE'],
 '2716992': ['rpoB-2'],
 '2716993': ['rpoB-1'],
 '2716994': ['psbF'],
 '2716995': ['psbL'],
 '2716996': ['petG'],
 '2716997': ['rps3'],
 '2716998': ['rpoC2'],
 '2717000': ['psaA'],
 '2717001': ['rps11'],
 '2717002': ['psbB'],
 '2717004': ['rpoA'],
 '2717005': ['rps2'],
 '2717006': ['rps2'],
 '2717007': ['rps18'],
 '271700

In [10]:
output_gene2names = "ncbigene2names_bigg_organisms_protein-coding.lzma"
with open(os.path.join(output_dir,output_gene2names), 'wb') as handle:
    compress_pickle.dump(geneid_to_names, handle, compression="lzma", set_default_extension=False)
print(f"GeneID to Names saved to {output_dir+output_gene2names}")

GeneID to Names saved to /Users/luna/Desktop/CRBM/AMAS_proj/AAAIM/data/ncbigene/ncbigene2names_bigg_organisms_protein_coding.lzma


### seperate by organism

In [14]:
output_geneinfo_bigg = "ncbi_gene_info_bigg_organisms_protein-coding.csv"
geneinfo_df = pd.read_csv(output_dir + output_geneinfo_bigg, dtype=str)

for tax_id in sorted(bigg_df['#tax_id'].unique()):
    organism_df = bigg_df[bigg_df['#tax_id'] == tax_id]
    geneid_to_names = {}
    for idx, row in organism_df.iterrows():
        gene_id = row['GeneID']
        symbol = row['Symbol'].strip() if pd.notnull(row['Symbol']) else ''
        synonyms = row['Synonyms'].strip() if pd.notnull(row['Synonyms']) else ''
        names = []
        if symbol and symbol != '-':
            names.append(symbol)
        if synonyms and synonyms != '-':
            # Split by '|' and strip whitespace
            names.extend([s.strip() for s in synonyms.split('|') if s.strip() and s.strip() != symbol])
        # Remove duplicates while preserving order
        seen = set()
        unique_names = []
        for n in names:
            if n not in seen:
                unique_names.append(n)
                seen.add(n)
        geneid_to_names[gene_id] = unique_names

    # Save organism-specific mapping
    output_gene2names = f"ncbigene2names_tax{tax_id}_protein-coding.lzma"
    
    with open(os.path.join(output_dir, output_gene2names), 'wb') as handle:
        compress_pickle.dump(geneid_to_names, handle, compression="lzma", set_default_extension=False)

    print(f"Created REF_NCBIGENE2NAMES for tax_id {tax_id}: {len(organism_df)} genes, "
            f"{len(geneid_to_names)} unique names -> {output_gene2names}")

Created REF_NCBIGENE2NAMES for tax_id 10029: 22956 genes, 22956 unique names -> ncbigene2names_tax10029_protein-coding.lzma
Created REF_NCBIGENE2NAMES for tax_id 10090: 26251 genes, 26251 unique names -> ncbigene2names_tax10090_protein-coding.lzma
Created REF_NCBIGENE2NAMES for tax_id 1120755: 5716 genes, 5716 unique names -> ncbigene2names_tax1120755_protein-coding.lzma
Created REF_NCBIGENE2NAMES for tax_id 126793: 3 genes, 3 unique names -> ncbigene2names_tax126793_protein-coding.lzma
Created REF_NCBIGENE2NAMES for tax_id 198214: 4313 genes, 4313 unique names -> ncbigene2names_tax198214_protein-coding.lzma
Created REF_NCBIGENE2NAMES for tax_id 224308: 4237 genes, 4237 unique names -> ncbigene2names_tax224308_protein-coding.lzma
Created REF_NCBIGENE2NAMES for tax_id 3055: 17819 genes, 17819 unique names -> ncbigene2names_tax3055_protein-coding.lzma
Created REF_NCBIGENE2NAMES for tax_id 344609: 30 genes, 30 unique names -> ncbigene2names_tax344609_protein-coding.lzma
Created REF_NCBIGE

## Create REF_NAMES2NCBIGENE

This is used for mapping all synonyms to ncbigene ids.

In [4]:
output_gene2names = "ncbigene2names_bigg_organisms_protein-coding.lzma"
geneid_to_names = compress_pickle.load(open(output_dir + output_gene2names, 'rb'))

In [5]:
exact_match_index = {}
for ncbigene, synonyms in geneid_to_names.items():
    for synonym in synonyms:
        # Create exact match index
        norm_synonym = synonym.lower()
        if norm_synonym not in exact_match_index:
            exact_match_index[norm_synonym] = []
        if ncbigene not in exact_match_index[norm_synonym]:
            exact_match_index[norm_synonym].append(ncbigene)
print(f"Exact match index has {len(exact_match_index)} entries")

Exact match index has 217047 entries


In [8]:
output_names2ncbigene = "names2ncbigene_bigg_organisms_protein-coding.lzma"
with open(os.path.join(output_dir, output_names2ncbigene), 'wb') as handle:
    compress_pickle.dump(exact_match_index, handle, compression="lzma", set_default_extension=False)
print(f"Names to NCBI Gene ID saved to {output_dir + output_names2ncbigene}")

Names to NCBI Gene ID saved to /Users/luna/Desktop/CRBM/AMAS_proj/AAAIM/data/ncbigene/names2ncbigene_bigg_organisms_protein_coding.lzma


### Separate REF_NAMES2NCBIGENE by organism
The organism-specific approach is necessary because the same gene name can refer 
to different gene IDs in different organisms.

In [10]:
output_geneinfo_bigg = "ncbi_gene_info_bigg_organisms_protein-coding.csv"
bigg_df = pd.read_csv(output_dir + output_geneinfo_bigg, dtype=str)

for tax_id in sorted(bigg_df['#tax_id'].unique()):
    organism_df = bigg_df[bigg_df['#tax_id'] == tax_id]
    
    # Create names to gene ID mapping for this organism
    names_to_geneid = {}
    
    for idx, row in organism_df.iterrows():
        gene_id = str(row['GeneID'])
        symbol = str(row['Symbol']).strip() if pd.notnull(row['Symbol']) else ''
        synonyms_str = str(row['Synonyms']).strip() if pd.notnull(row['Synonyms']) else ''
        
        # Collect all names for this gene
        names = []
        if symbol and symbol != '-' and symbol != 'nan':
            names.append(symbol)
        if synonyms_str and synonyms_str != '-' and synonyms_str != 'nan':
            # Split by '|' and strip whitespace
            names.extend([s.strip() for s in synonyms_str.split('|') 
                        if s.strip() and s.strip() != symbol and s.strip() != '-'])
        
        # Add each name to the mapping (lowercase for case-insensitive lookup)
        for name in names:
            if name:
                norm_name = name.lower()
                if norm_name not in names_to_geneid:
                    names_to_geneid[norm_name] = []
                if gene_id not in names_to_geneid[norm_name]:
                    names_to_geneid[norm_name].append(gene_id)
    
    # Save organism-specific mapping
    output_names2gene = f"names2ncbigene_tax{tax_id}_protein-coding.lzma"
    
    with open(os.path.join(output_dir, output_names2gene), 'wb') as handle:
        compress_pickle.dump(names_to_geneid, handle, compression="lzma", set_default_extension=False)
   
    print(f"Created REF_NAMES2NCBIGENE for tax_id {tax_id}: {len(organism_df)} genes, "
            f"{len(names_to_geneid)} unique names -> {output_names2gene}")

Created REF_NAMES2NCBIGENE for tax_id 10029: 22956 genes, 25483 unique names -> names2ncbigene_tax10029_protein-coding.lzma
Created REF_NAMES2NCBIGENE for tax_id 10090: 26251 genes, 77266 unique names -> names2ncbigene_tax10090_protein-coding.lzma
Created REF_NAMES2NCBIGENE for tax_id 1120755: 5716 genes, 5716 unique names -> names2ncbigene_tax1120755_protein-coding.lzma
Created REF_NAMES2NCBIGENE for tax_id 126793: 3 genes, 3 unique names -> names2ncbigene_tax126793_protein-coding.lzma
Created REF_NAMES2NCBIGENE for tax_id 198214: 4313 genes, 4102 unique names -> names2ncbigene_tax198214_protein-coding.lzma
Created REF_NAMES2NCBIGENE for tax_id 224308: 4237 genes, 8468 unique names -> names2ncbigene_tax224308_protein-coding.lzma
Created REF_NAMES2NCBIGENE for tax_id 3055: 17819 genes, 30361 unique names -> names2ncbigene_tax3055_protein-coding.lzma
Created REF_NAMES2NCBIGENE for tax_id 344609: 30 genes, 48 unique names -> names2ncbigene_tax344609_protein-coding.lzma
Created REF_NAMES2

## Add organisms to NCBI gene reference files

In [12]:
from add_organism_by_taxid import add_organism_data, get_organism_info
data_dir = '/Users/luna/Desktop/CRBM/AMAS_proj/Data/'

# first, get the organism info
tax_id = ['83332'] # Mycobacterium tuberculosis (H37Rv)
info_df = get_organism_info(tax_id)

Querying tax_ids: ['83332']
Loading from: /Users/luna/Desktop/CRBM/AMAS_proj/Data/ncbi/All_Data.gene_info.gz

Found data for 1 organisms:
tax_id      gene_type  gene_count  unique_symbols                           sample_genes
 83332        miscRNA           2               2                              rnpB, ssr
 83332          ncRNA          20              20      B55, mcr10, mcr11, mcr15, MTS1082
 83332          other           3               3                 Rv2280, kgtP, NEWENTRY
 83332 protein-coding        3906            3906        mprA, dppD, trpE, dnaA, Rv0913c
 83332         pseudo          30              30 Rv3636, Rv0947c, Rv1667c, Rv1150, esxM
 83332           rRNA           3               3                          rrs, rrf, rrl
 83332           tRNA          45              45           serT, argT, argW, leuW, tyrT


In [15]:
# Adding to reference files
result = add_organism_data(['83332'], output_suffix="_updated", data_dir=data_dir)

Data directory: /Users/luna/Desktop/CRBM/AMAS_proj/Data
Output directory: /Users/luna/Desktop/CRBM/AMAS_proj/AAAIM/data/ncbigene
Gene info file: /Users/luna/Desktop/CRBM/AMAS_proj/Data/ncbi/All_Data.gene_info.gz
Adding organisms with tax_ids: ['83332']
Loading gene info data...
Found 3906 genes across 1 organisms
Organisms found: ['83332']
Created names2ncbigene_tax83332_protein-coding_updated.lzma: 3906 genes, 4268 unique names
Loading existing label file: /Users/luna/Desktop/CRBM/AMAS_proj/AAAIM/data/ncbigene/ncbigene2label_bigg_organisms_protein-coding.lzma
Updated global label file: ncbigene2label_bigg_organisms_protein-coding_updated.lzma (150748 total entries)

=== Summary ===
Created 2 files for 1 organisms
Total genes added: 3906
Total unique names: 4268


In [3]:
from add_organism_by_taxid import add_organism_data, get_organism_info
data_dir = '/Users/luna/Desktop/CRBM/AMAS_proj/Data/'

# first, get the organism info
tax_id = ['227984'] 
info_df = get_organism_info(tax_id)

Loading gene info from: /Users/luna/Desktop/CRBM/AMAS_proj/Data/ncbi/All_Data.gene_info.gz

Organism gene counts:
type_of_gene  other  protein-coding  total
#tax_id                                   
227984            1              13     14


In [4]:
# Adding to reference files
result = add_organism_data(['227984'], output_suffix="_added", data_dir=data_dir)

Adding organisms with tax_ids: ['227984']
Loading gene info from: /Users/luna/Desktop/CRBM/AMAS_proj/Data/ncbi/All_Data.gene_info.gz
Found 13 genes for tax_id 227984
Created files for tax_id 227984: 13 genes, 14 unique names
  - names2ncbigene_tax227984_protein-coding.lzma (for search)
  - ncbigene2names_tax227984_protein-coding.lzma (for RAG embeddings)
Loading existing label file: /Users/luna/Desktop/CRBM/AMAS_proj/AAAIM/data/ncbigene/ncbigene2label_bigg_organisms_protein-coding.lzma
Existing labels: 146842 genes
After adding new organisms: 146855 genes
Updated global label file: ncbigene2label_bigg_organisms_protein-coding_added.lzma

=== Summary ===
Added 1 organisms
Total genes: 13
Created/updated 3 files:
/Users/luna/Desktop/CRBM/AMAS_proj/AAAIM/data/ncbigene/names2ncbigene_tax227984_protein-coding.lzma
/Users/luna/Desktop/CRBM/AMAS_proj/AAAIM/data/ncbigene/ncbigene2names_tax227984_protein-coding.lzma
/Users/luna/Desktop/CRBM/AMAS_proj/AAAIM/data/ncbigene/ncbigene2label_bigg_orga

## Save descriptions for RAG embeddings

In [ ]:
# All gene data with description
input_file = DATA_DIR + "ncbi/All_Data.gene_info.gz"
columns_to_keep = ["#tax_id", "GeneID", "Symbol", "Synonyms","type_of_gene","description"]
df_des = pd.read_csv(input_file, compression='gzip', sep='\t',usecols=columns_to_keep)
df_des.head()